<a href="https://colab.research.google.com/github/osaidameer/Book_Recommendation_KNN/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2023-04-04 23:52:07--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.6’

book-crossings.zip. 100%[===================>]  24.88M   132MB/s    in 0.2s    

2023-04-04 23:52:07 (132 MB/s) - ‘book-crossings.zip.6’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace BX-Books.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace BX-Users.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# add your code here - consider creating a new cell for each section of code
# remove from the dataset users with less than 200 ratings and books with less than 100 ratings.
# cleaning data

list_books = df_ratings['isbn'].value_counts()
list_users = df_ratings['user'].value_counts()

users_to_drop = list_users[list_users < 200].index
books_to_drop = list_books[list_books < 100].index

df_ratings = df_ratings.loc[~(df_ratings['user'].isin(users_to_drop) | df_ratings['isbn'].isin(books_to_drop))]
df_books = df_books.loc[~(df_books['isbn'].isin(books_to_drop))]

(1149780, 3) (271379, 3)
(49781, 3) (1936, 3)


In [ ]:
# merge and pivot data
df_books_ratings = pd.merge(df_ratings, df_books, on='isbn', how='inner').drop_duplicates(['title', 'user'])
df_pivoted_books_ratings = df_books_ratings.pivot_table(index='title', columns='user', values='rating').fillna(0)

(49136, 5)


(673, 888)

In [ ]:
# initialise knn
nn = NearestNeighbors(metric='cosine', algorithm='brute')
nn.fit(df_pivoted_books_ratings.values)

NearestNeighbors(algorithm='brute', metric='cosine')

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  distances, indices = nn.kneighbors(df_pivoted_books_ratings.loc[book].values.reshape(1, -1), n_neighbors=6)
  distances, indices = np.flip(distances)[0], np.flip(indices)[0]
  recommended_books = [book, [[df_pivoted_books_ratings.iloc[indices[index]].name, distances[index]] for index in range(0, len(distances) - 1)]]
  return recommended_books

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
